# Sequence-to-Sequence Models
<hr>

We have learned that RNNs with a many-to-many architecture (with encode-decoder networks as a special form) take a sequence as an input and also produce a sequence as output. Such models are called **Sequence-to-Sequence (S2S) models.** Such networks are traditionally used in machine translation where the input consists of sentences, which are transformed by an encoder network to serve as the input for the decoder network which does the actual translation. The output is then again a sequence, namely the translated sentence. The same process works likewise for other data. You could for instance take an image as input and have an RNN try to produce a sentence that states what is on the picture (image captioning). The encoder network in this case could then be a conventional CNN whereas the last layer will contain the image as a vector. This vector is then served to an RNN which acts as the decoder network to make predictions. Assuming you have enough already captioned images as training data an RNN could then learn how to produce captions for yet unseen images.

## 1 - S2S in Machine Translation

There are certain similarities between **S2S-models** and the previously seen language-models where we had an RNN produce a sequence of words based on the probability of previously seen words in the sequence:

<br>

<div style="text-align:center">
    <img src="images/language-model.png" width=300>
    <center><caption><font color="purple"><b>Figure 1:</b> Structure of a language model</font></caption></center>
</div>

In such a setting, the output was generated by producing a somewhat random sequence of words. However, that is not what we want in machine translation. Here we usually want the most likely sentence that corresponds to the best translation for a given input sentence. This is a key difference between language models as seen before and machine translation models. So in contrast to the model above, a machine translation model does not start with the zero vector as the first token, but rather takes a whole sentence and encodes it using the encoder part of the network.

<br>

<div style="text-align:center">
    <img src="images/machine-translation.png" width=450>
    <center><caption><font color="purple"><b>Figure 2:</b> Structure of a machine translation model</font></caption></center>
</div>

In that respect one can think of a machine translation model as kind of **conditional language model** that calculates the probability of a translated sentence given the input sentence in the original language: $P$( translation $\mid$ input language)

## 2 - Beam Search

Suppose we have the following french sentence as an input:

```
Jane visite l'Afrique en septembre.
```

Possible translations to English for this sentence could be:

```
- Jane is visiting Africa in September.
- Jane is going to be visiting Africa in September.
- In September, Jane will visit Africa.
- Her African friend welcomed Jane in September.
```

Each of these sentences is a more or less adequate translation of the input sentence. One possible approach to calculate the most likely sentence (i.e. the best translation) is going through the words one by one to calculate the joint probability of a sentence by always taking the most probable word in each step as the next token. This approach is called **greedy search** and although it is fairly simple, it does not work well for machine translation. This is because small differences in probability for earlier words can have a big impact on what sentence is picked after inspection of all tokens. By using greedy search we would also try to find the best translation in a set of exponentially many sentences. Considering a vocabulary of 10,000 words and a sentence length of 10 words this would yield a solution space of $10,000^{10}$ theoretically possible sentences.

If the algorithm has picked up the first 2 words "Jane is" and is deciding on the 3rd word, then 'going' is likely to be more probable than 'visiting' and thus the algorithm might pick "Jane is going" as the first three letters which might not be the best choice of words in the translation.

Another approach which works reasonably well is **beam search (BS).** BS is an approximative approach, i.e. it tries to find a good enough solution. This has computational advantages over exact approaches like `BFS` or `DFS`, meaning the algorithm runs much faster but does not guarantee to find the exact optimum.

BS also goes through the sequence of words one by one. But instead of choosing the one most likely word as the next token in each step as greedy does, it considers a **number of alternatives** at the same time. This **number $B$** is called the **beam width.** In each step, only the $B$ most likely choices are kept as the next word. This means that a partial sequance is not further evaluated if it does not continue with one of the $B$ most likely words. The following figure illustrates this:

<br>

<div style="text-align:center">
    <img src="images/beam-search.png" width=850>
    <center><caption><font color="purple"><b>Figure 3:</b> Beam search for $B=3$</font></caption></center>
</div>

The higher the value for $B$ the more alternatives are considered in each step, hence the more combinatorial paths are followed and the computationally more expensive the algorithm becomes. With a value of $B=1$, BS essentially becomes greedy search. Productive systems usually choose a value between 10 and 100 for $B$. Larger values like $1000 < B < 3000$ happen to be used for research purposes, although the final utility will decrease the bigger $B$ becomes.

## 2.1 - Refinements to Beam Search

There are a few tricks that help improve BS performance:

- Length normalization
- Error analysis

### 2.1.1 - Length normalization
We have seen BS maximizing the following probability:

$$
\begin{align}
\text{translation} &= \text{arg max} \prod_{t=1}^{T_y} P\left( y^{<t>} \mid x, y^{<1>}, \cdots, y^{<t-1>}\right) \\
&= P \left( y^{<1>}, \cdots, y^{<T_y>} \mid x \right) \\
&= P\left( y^{<1>} \mid x \right) \cdot P\left( y^{<2>} \mid x, y^{<1>} \right) \cdots P\left( y^{<T_y>} \mid x, y^{<1>}, \cdots, y^{<T_y -1 >} \right)
\end{align}
$$

The problem with this is that the probabilities of the individual tokens are usually much smaller than 1 which results in an overall probability which might be too small to be stored by a computer (numerical underflow). Therefore, we use the **log-likelihood** of the probabilities:

$$
\text{arg max} \sum_{t=1}^{T_y} \log{ P\left( y^{<t>} \mid x, y^{<1>}, \cdots, y^{<t-1>}\right)}
$$

Because the logarithm is a strictly monotonically increasing function maximizing the product of probabilities is the same as maximizing the sum of their logarithms. However, this formula will still result in shorter sequences because the overall probability for a sentence will only decrease the longer it gets. Longer sequences are therefore penalized while shorter sequences benefit from the higher probability. To fix this, a normalization parameter $\alpha$ is introduced and the term is normalized by dividing by a power of the number of words in the sequence (length normalization):

$$
\text{arg max} \frac{1}{T_y ^ \alpha} \sum_{t=1}^{T_y} \log{ P\left( y^{<t>} \mid x, y^{<1>}, \cdots, y^{<t-1>}\right)}
$$

The hyperparamter $\alpha$ controls the degree of normalization. This means instead of comparing the raw values for the probability of a partial sequence, Beam Search will compare the normalized values. For values of $\alpha \rightarrow 1$, the term is completely normalized, for values of $\alpha \rightarrow 0$ no normalization is done.

### 2.1.2 - Error Analysis
If a system does not output sequences with the desired quality, we have to examine where causes for this are possibly rooted: in the RNN model itself or in the Beam Search algorithm.

Consider the French sentence "Jane visite l'Afrique en septembre" from above. We now compare this sentence to the following sentences:

- $y^*$: optimal translation by a human (e.g. "Jane visits Africa in September.")
- $\hat{y}$: translation from the model (e.g. "Jane visited Africa last September.")

In this example it is evident that the model did not output a good translation. We can now distinguish the following cases for the probabilities of those two sentences (from the set of all possible sentences).

- $P\left( y^* \mid x \right) > P\left( \hat{y} \mid x \right)$: In this case the algorithm chose the sentence with the lower probability. The cause for the error is therefore rooted in the search algorithm: <font color="red">Error in Beam Search.</font>

- $P\left( y^* \mid x \right) \leq P\left( \hat{y} \mid x \right)$: In this case the algorithm calculated a too small probability for the optimal translation. The cause for the error is therefore rooted in the model: <font color="red">Error in RNN model.</font>

To get a feeling for where most errors are rooted we could make this comparison for a number of samples and analyze where most errors are rooted. If the fraction of errors for the RNN are greater than those for the Beam Search, we'd want to invest more time in fixing the RNN output either by collecting more data, normalization, etc.

## 3 - Evaluating Machine Translation 

A special property of translation is that there are possibly many different translation which are considered equally correct. Since linguistics is not an exact science and language itself is somewhat fuzzy due to its ambiguity, the evaluation of NLP tasks naturally contains a grey area where the distinction between correct or wrong is not clear.

One method to compare two texts (e.g. human and machine translation) is to calculate the Bleu-Score (Bilingual evaluation understudy). This value measures the quality of a translation as the degree of overlap with a (or several) reference translation(s). A higher value means a better quality.

### 3.1 - Bilingual Evaluation Understudy (Blue) Score

The Bleu score measures precision as the number of words in the translation that also appear in the reference. Consider the following translations for the French sentence "Le chat es sur le tapis."

```
Reference translation 1:	The cat is on the mat.
Reference translation 2:	There is a cat on the mat.
Translation:	the the the the the the the (7 words)
```

Because the word `the` appears in at least one of the reference translation, the resulting Bleu-score would be $\frac{7}{7}=1$ (i.e. a perfect translation). However, this is obviously wrong. Therefore a modified precision is used which limits the number of counted appearances to the maximum number of appearances in either one of the sentences. Because in the above example the word `the` appears twice in the first reference translation and once in the second reference translation, this maximum is 2 and the modified precision is therefore $\frac{2}{7}$.

Instead of looking at words in isolation (unigrams) we could also look at pairs of words (bigrams), trigrams (3-grams) or generally tuples of any number (n-grams). For this, the machine translation sentence is split into its different n-grams which are then counted. We could then additionally calculate a clip count which indicates in how many of the reference sentences the n-gram appears.

| Bigram | Count | Clip Count |
| ---- | ---- | ---- |
| the cat | 2 | 1 |
| cat the | 1 | 0 |
| cat on | 1 | 1 |
| on the | 1 | 1 |
| the mat | 1 | 1 |

The modified n-gram-precision could then be calculated by dividing the sum of clipped counts by the sum of all counts:

$$p_n = \frac{\sum_{\text{n-gram} \in \hat{y}} count_{\text{clip}} (n-gram)}{\sum_{\text{n-gram} \in \hat{y}} count (n-gram)}$$

For the above example, the $p_2$ value would be $\frac{4}{6} \approx 0.666$. A good translation has values for different n-grams close to 1. All n-gram-scores can be combined to the combined Bleu score:

$$exp \left( \frac{1}{4} \sum_{n=1}^4 p_n \right)$$

Of course, again short translations have an advantage here because there are fewer possibilities for errors. An extreme example of this would be a translation in form of a single word which would get a score of 1 if only this words appears in the reference translations. To prevent this, an adjustment factor called **brevity penalty (BP)** is introduced, which penalizes short translations:

```
MT = Machine Translation
RT = Reference (human) Translation
```

$$BP = \begin{cases}
1 & \text{if MT_length > RT_length} \\
exp \left(1 - \frac{RT_{\text{length}}}{MT_{\text{length}}} \right) & \text{otherwise} \\
\end{cases}
$$